In [ ]:
import pandas as pd

In [ ]:
import json

In [ ]:
import ast

In [ ]:
import re  # Importando a biblioteca de expressões regulares

In [ ]:
!pip install openai==0.28

In [ ]:
import openai

In [ ]:
openai.api_key = "Chave de acesso"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('caminho/base.csv')

In [ ]:
df

In [ ]:
print(df.classe.value_counts())

In [ ]:
df['classe'] = df['classe'].replace({'Negativo': -1, 'Positivo': 1, 'Neutro': 0})

In [ ]:
print(df.classe.value_counts())

In [ ]:
df

In [ ]:
df['gptpt'] = ''

In [ ]:
df

In [ ]:
df['descricaoGptpt'] = ''

In [ ]:
df

In [ ]:
df['erro'] = ''

In [ ]:
df

In [ ]:
def get_completion(prompt, model):
  try:
    # formatar o prompt como uma mensagem de usuário para a API de chat
    messages = [{"role": "user", "content": prompt}]

    # chamar a API de chat com o modelo especificado, as mensagens e a temperatura
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    # retornar o conteúdo da resposta
    return response.choices[0].message["content"]
  except Exception as e:
        return "erro"

In [ ]:
model="gpt-3.5-turbo"

for index, row in df.iterrows():
  try:
        frase = row['tweet']
        empresa = row['EMPRESA']

        inicio_prompt = "Você é um profissional de marketing da empresa '" + empresa + "'. Classifique a frase como 'positiva', 'negativa' ou 'neutra'. A frase pode conter sarcasmo, ironia ou polarity shift. A frase deve ser analisada do ponto de vista da empresa '" + empresa + "'. "\
        "Por fim, explique como você chegou a essas conclusões. Retorne todas essas informações no formato de um objeto JSON: { 'classe': 'classe', 'explicacao_modelo': 'string'} "\

        prompt_final = inicio_prompt + " Frase a ser classificada: "\
        "Frase: '" + frase + "' "\
                "JSON esperado:"\
                "{" \
                   "'classe': 'classe'," \
                    "'explicacao_modelo': 'string'" \
                "}"

        print(prompt_final)

        response = get_completion(prompt_final, model)

        print(response)

        response_dict = json.loads(response)

        # Salve o sentimento na coluna correspondente
        df.at[index, 'gptpt'] = response_dict['classe']
        df.at[index, 'descricaoGptpt'] = response_dict['explicacao_modelo'].replace("'", "")

        print("----------------------------------------------------------------------------------------------------------------------------------")
  except Exception as e:
      df.at[index, 'erro'] = response

In [ ]:
df

In [ ]:
print(df.erro.value_counts())

In [ ]:
df

In [ ]:
df.to_csv('caminho/base.csv', index=False)

In [ ]:
print(df.classe.value_counts())

In [ ]:
print(df.gptpt.value_counts())

In [ ]:
df['gptpt'] = df['gptpt'].replace({'negativa': -1, 'positiva': 1, 'neutra':0})

In [ ]:
print(df.gptpt.value_counts())

In [ ]:
df

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def metricas(classe, gpt):
  # Calcular as métricas
  accuracy = accuracy_score(classe, gpt)
  precision = precision_score(classe, gpt, average='weighted')
  recall = recall_score(classe, gpt, average='weighted')
  f1 = f1_score(classe, gpt, average='weighted')
  print("Accuracy:", accuracy)
  print("Precision:", precision)
  print("Recall:", recall)
  print("F1-score:", f1)

In [ ]:
metricas(df['classe'], df['gptpt'])

In [ ]:
df_diff = df[df['classe'] != df['gptpt']]

In [ ]:
df_diff.to_csv('/content/drive/MyDrive/dados dissertacao/rotulação/bases jonn/Mining_BR/TESTE_Planilha_fewshot_colunas_diferencas_zeroshot_contexto.csv', index=False)

In [ ]:
df_diff